In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))
print("UDP Server is ready to receive API data...")

clients = {}  # Store authenticated clients as {address: client_id}
valid_clients = {'client1': 'password1', 'client2': 'password2'}

def authenticate(data):
    try:
        client_id, password, message = data.decode().split(':', 2)
        if valid_clients.get(client_id) == password:
            return True, client_id, message
        else:
            return False, None, None
    except Exception as e:
        print(f"Authentication error: {e}")
        return False, None, None

while True:
    data, client_address = server_socket.recvfrom(2048)
    print(f"Received data from {client_address}: {data.decode()}")

    is_authenticated, client_id, message = authenticate(data)
    if not is_authenticated:
        server_socket.sendto(b"Authentication failed", client_address)
        print(f"Authentication failed for {client_address}")
        continue

    # Add to clients if new
    if client_address not in clients:
        clients[client_address] = client_id
        print(f"Added {client_address} ({client_id}) to the list of clients")

    # Send acknowledgment back to sender
    server_socket.sendto(f"Message received: {message}".encode(), client_address)

    # Forward message to all other clients
    for client in clients:
        if client != client_address:
            server_socket.sendto(f"{client_id}: {message}".encode(), client)


UDP Server is ready to receive API data...


In [ ]:
import socket
import requests
# Fetch weather data
api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.47&longitude=-0.0363&current_weather=true"
response = requests.get(api_url)
if response.status_code == 200:
    weather_data = response.json()
    temperature = weather_data["current_weather"]["temperature"]
    message = f"Current temperature: {temperature}°C"
else:
    message = "Failed to fetch weather data"
# Send the weather data using UDP
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)
client_socket.sendto(message.encode(), server_address)
print("Weather data sent!")
client_socket.close()

In [ ]:
import socket

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)

client_id = 'client1'
password = 'password1'

while True:
    message = input("Enter message to send: ")
    auth_message = f"{client_id}:{password}:{message}"
    client_socket.sendto(auth_message.encode(), server_address)
    
    if message == "exit":
        break

    try:
        data, _ = client_socket.recvfrom(2048)
        response = data.decode()
        print(f"Received response from server: {response}")
    except socket.timeout:
        print("No response from server.")

client_socket.close()


Received response from server: Message received: bob
